# Evaluation Metrics

In [1]:
# Import some helper functions (please ignore this!)
from utils import *

**Context:**
* In our regression and classification models, we've used 1- and 2-dimensional input data and 1-dimensional output data---data that we can easily visualize. This enabled us to understand how well our model fits qualitatively.
* But for many real-life ML tasks, the data is very high dimensional. Give examples.

**Challenge:**

**Outline:**

**Data:**
* Introduce motivating high-dimensional regression and classification data and tasks.
* Provide them students with several choices of fitted models for each task to evaluate.

## Preventing Over- and Under-fitting

TODO:
* Explain both concepts, show visuals for regression and classification
* Introduce the idea of validation and test set
* Raise question: but which metric to use?

## Log-Likelihood

TODO:
* Pros: can be used on all probabilistic models
* Pros: has intuitive properties (exercise with regression with biased mean, and with biased variance)
* Cons: units are difficult to interpret
* Note: which points you include in your test set matters

## Metrics for Regression

TODO:
* Introduce MSE
* Pros: easy to interpret
* Cons: hard to know if scale of MSE is reasonable
* Exercise: ask students to compute MSE and log-likelihood and decide which model they prefer
* Exercise: add additional sensitive feature to data---ask them to break down metrics on sensitive variable. Which model do they pick now? 

## Metrics for Classification

TODO:
* Introduce accuracy. Con: doesn't communicate false positives of negatives (use class imbalance example)
* Introduce confusion matrix. Cons: doesn't communicate predictive uncertainty
* Exercise: ask students to compute confusion matrices and decide which model they prefer
* Exercise: add an additional sensitive feature to data---ask them to break down metrics on sensitive variable. Which model do they pick now? 

## Challenges with Evaluation

TODO:
* Model performance is multi-dimensional, but we can only use one "scoring system" to select
* This is becomes even more tricky when fairness is involved
* How to be ethical? Participatory design.